In [9]:
import os
import torch
import cv2
from pathlib import Path
from openpyxl import Workbook
import matplotlib.pyplot as plt
from ultralytics import YOLO
import numpy as np


In [10]:

# model = YOLO("best.pt")  


# results = model('word_key_yolo_used',save=True)

# print(results)

model = YOLO('best.pt')

# 指定圖片資料夾並獲取所有圖片路徑
image_dir = 'word_key_yolo_used'
image_paths = [os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith(('.png', '.jpg', '.jpeg'))]

# 執行模型進行偵測
results = model(image_paths)  # 使用圖片路徑列表進行偵測

# 創建一個資料夾來儲存有 bounding box 的圖片
save_dir = 'detected_images'
os.makedirs(save_dir, exist_ok=True)

# 處理結果列表
for i, (result, img_path) in enumerate(zip(results, image_paths)):
    image_name = Path(img_path).name  # 使用 Path 來獲取檔案名
    detections = []

    # 遍歷所有偵測到的 bounding boxes
    if result.boxes:
        boxes = result.boxes.data  # 獲取包含座標和類別信息的 tensor
        for box in boxes:
            class_id = int(box[5])  # 假設類別索引在第六列
            confidence = box[4]     # 假設置信度在第五列
            class_name = model.names[class_id]  # 獲取類別名稱
            detections.append(f"{class_name} ({confidence:.2f})")

    # 打印圖片名稱和所有偵測結果
    print(f"Image: {image_name}, Detections: {', '.join(detections)}")

    # 保存有 bounding box 的圖片
    save_path = os.path.join(save_dir, f'key_{i}.jpg')
    result.save(save_path)  # 假設 result.save() 方法存在


#####HOW to use YOLO V8 inference(official)
# from ultralytics import YOLO

# # Load a model
# model = YOLO('yolov8n.pt')  # pretrained YOLOv8n model

# # Run batched inference on a list of images
# results = model(['im1.jpg', 'im2.jpg'])  # return a list of Results objects

# # Process results list
# for result in results:
#     boxes = result.boxes  # Boxes object for bounding box outputs
#     masks = result.masks  # Masks object for segmentation masks outputs
#     keypoints = result.keypoints  # Keypoints object for pose outputs
#     probs = result.probs  # Probs object for classification outputs
#     obb = result.obb  # Oriented boxes object for OBB outputs
#     result.show()  # display to screen
#     result.save(filename='result.jpg')  # save to disk


0: 640x640 1 right, 5.1ms
1: 640x640 1 Ctrl, 5.1ms
2: 640x640 1 Action 4, 1 Page Down key, 5.1ms
3: 640x640 1 up, 5.1ms
4: 640x640 1 shift, 5.1ms
5: 640x640 1 Question mark, 1 forwar_slash, 5.1ms
6: 640x640 1 Greater than, 1 Less than, 1 dot, 5.1ms
7: 640x640 1 Greater than, 1 Less than, 1 comma, 5.1ms
8: 640x640 1 M, 5.1ms
9: 640x640 1 N, 5.1ms
10: 640x640 1 B, 5.1ms
11: 640x640 (no detections), 5.1ms
12: 640x640 1 key_board, 5.1ms
13: 640x640 1 C, 5.1ms
14: 640x640 1 X, 5.1ms
15: 640x640 1 2, 5.1ms
16: 640x640 1 shift, 5.1ms
17: 640x640 1 Action 3, 1 Page Up key, 5.1ms
18: 640x640 1 enter, 5.1ms
19: 640x640 2 Quotes, 1 Single quote, 5.1ms
20: 640x640 1 Colon, 1 Semicolon, 5.1ms
21: 640x640 1 L, 5.1ms
22: 640x640 1 right, 5.1ms
23: 640x640 1 K, 5.1ms
24: 640x640 1 J, 5.1ms
25: 640x640 1 H, 5.1ms
26: 640x640 1 G, 5.1ms
27: 640x640 1 F, 5.1ms
28: 640x640 1 D, 5.1ms
29: 640x640 1 S, 5.1ms
30: 640x640 1 A, 5.1ms
31: 640x640 1 Caps Lock, 5.1ms
32: 640x640 1 Action 2, 1 end, 5.1ms
33: 640x

In [35]:
import os
from pathlib import Path
import cv2
from openpyxl import Workbook
from openpyxl.drawing.image import Image as OpenpyxlImage
from openpyxl.utils import get_column_letter
from ultralytics import YOLO

# 初始化 YOLO 模型
model = YOLO('best.pt')

# 初始化工作簿和工作表
wb = Workbook()
ws = wb.active
ws.title = "YOLO Detection Results"

# 设置列宽
columns_width = [40, 25, 65, 25, 65]  # 根据需要调整列宽
for i, width in enumerate(columns_width, start=1):
    ws.column_dimensions[get_column_letter(i)].width = width

# 添加列标题
column_titles = ["Compare Result", "Key Image 1", "YOLO Image 1 Result", "Key Image 2", "YOLO Image 2 Result"]
for i, title in enumerate(column_titles, start=1):
    ws.cell(row=1, column=i).value = title

# 定义文件夹
yolo_dirs = ['./test1_used_image/result_file', './test2_used_image/result_file']
save_dirs = ['./detected_images1', './detected_images2']
compare_dir = './compare_result'
for dir in save_dirs + [compare_dir]:
    os.makedirs(dir, exist_ok=True)

# 暂存临时图片路径，用于稍后删除
temp_image_paths = []

# 初始化行数
row = 2

# 处理每对图片
for image_files in zip(os.listdir(yolo_dirs[0]), os.listdir(yolo_dirs[1]), os.listdir(compare_dir)):
    compare_image_path = os.path.join(compare_dir, image_files[2])
    compare_image = cv2.imread(compare_image_path)
    if compare_image is None:
        print(f"Failed to load compare image {image_files[2]}")
        continue

    # 处理比较图片
    temp_compare_path = f"temp_compare_{row}.png"
    cv2.imwrite(temp_compare_path, compare_image)
    temp_image_paths.append(temp_compare_path)
    compare_img = OpenpyxlImage(temp_compare_path)
    compare_img.anchor = ws.cell(row, 1).coordinate
    compare_img.width, compare_img.height = 210, 90
    ws.add_image(compare_img)

    for idx, (yolo_dir, save_dir, image_file) in enumerate(zip(yolo_dirs, save_dirs, image_files[:2])):
        yolo_image_path = os.path.join(yolo_dir, image_file)
        yolo_image = cv2.imread(yolo_image_path)

        if yolo_image is None:
            print(f"Failed to load image for YOLO {image_file}")
            continue

        # 进行YOLO检测
        results = model([yolo_image_path])
        result = results[0] if results else None
        detections = []
        if result and result.boxes:
            for box in result.boxes.data:
                class_id = int(box[5])
                confidence = box[4]
                class_name = model.names[class_id]
                detections.append(f"{class_name} ({confidence:.2f})")

        # 保存YOLO检测结果图片
        detected_image_path = os.path.join(save_dir, f'detected_{Path(image_file).stem}.jpg')
        result.save(detected_image_path)

        # 将YOLO检测结果图片添加到Excel，并调整图片大小
        yolo_img = OpenpyxlImage(detected_image_path)
        yolo_img.anchor = ws.cell(row, 2 + idx * 2).coordinate
        yolo_img.width, yolo_img.height = 90, 90  # 调整YOLO图片大小
        ws.add_image(yolo_img)

        # 将YOLO文字结果写入Excel
        ws.cell(row=row, column=3 + idx * 2).value = ", ".join(detections)

    # 调整行高
    ws.row_dimensions[row].height = 67.5

    # 行数增加
    row += 1

# 保存工作簿
wb.save("YOLO_Detection_Results.xlsx")
print("Excel file saved as 'YOLO_Detection_Results.xlsx'")

# 删除所有临时图片
for temp_path in temp_image_paths:
    os.remove(temp_path)









0: 640x640 1 K, 90.3ms
Speed: 5.1ms preprocess, 90.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 K, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 D, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 D, 7.0ms
Speed: 2.0ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Brightness Up, 7.0ms
Speed: 2.5ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Brightness Up, 1 Dictation, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Brightness Down, 1 Insert Key, 8.0ms
Speed: 2.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Insert Key, 1 Search Spotlight, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.5ms 